# Step 1: Query the ASF Database
This notebook uses the dinosar python package to search on the ASF (Alaksa Satellite Facility) for Sentinel 1-A/B SAR data convering the APMB (Altiplano-Puna Magma Body) for all times.

## 1.1: Import Python Modules
Import the python modules you need (and install dinosar if necessary)

In [ ]:
# if dinosar library not in base environment uncomment below (run just once)
#!pip install --no-cache git+https://github.com/scottyhq/dinosar.git@master

In [ ]:
import geopandas as gpd
import pandas as pd
import geoviews as gv
import hvplot.pandas
import dinosar

## 1.2: Load and plot area of interest

Load the file "apmb.geojson" that contains the bounding coordinates for the APMB, our area of interest (AOI).  Make a plot to confirm everything worked.

In [ ]:
gf = gpd.read_file('apmb.geojson') #load file with apmb coordinates
snwe = dinosar.archive.asf.ogr2snwe('apmb.geojson') #grad the lat/lon extent of the apmb
snwe #print out the lat/lon extent

In [ ]:
# Plot the area of interest on a map

S,N,W,E = [-25, -20, -72, -65] #set map extent

# set up plot elements
tiles = gv.tile_sources.CartoEco.options(width=700, height=500).redim.range(Latitude=(S, N), Longitude=(W, E)) 
aoi = gf.hvplot(geo=True, fill_color=None, line_color='orange', title='Sentinel-1 Area of Interest', hover=False)
utur = gv.Points([(-67.18, -22.27)]).opts(color='r', marker='^', size=10)

tiles * aoi * utur #This line actually displays the plot

## 1.3: Query the ASF database and plot results
Search for Sentinel 1-A/B data on ASF that falls within the lat/lon extents defined by our apmb.geojson file.  Plot the results of the search.

In [ ]:
# Define a function that queries the ASF database for both SA and SB results, 
# and merges search results into one file ("query.geojson").
# If "query.geojson" already exists, this function will overwrite it with a new file 
# with updated search results.
def refresh_inventory(snwe):
    dinosar.archive.asf.query_asf(snwe, 'SA') #creates the file "query_SA.json"
    dinosar.archive.asf.query_asf(snwe, 'SB') #creates the file "query_SB.json"
    gf = dinosar.archive.asf.merge_inventories('query_SA.json', 'query_SB.json')#merge the results 
    dinosar.archive.asf.save_inventory(gf)#creates the merged file "query.geojson"

In [ ]:
refresh_inventory(snwe) #refresh existing file or create a new one

In [ ]:
#load the search results
gf = dinosar.archive.asf.load_inventory('query.geojson')

In [ ]:
# print the track/orbit numbers in the search data
gf.relativeOrbit.unique()

In [ ]:
# display a table summarizing the geometries of the different orbits
gft = gf[['relativeOrbit','geometry']].dissolve(by='relativeOrbit').reset_index()
gft['code'] = gft.relativeOrbit.astype('category').cat.codes # sets a code for orbits
gft.head()

In [ ]:
# get geometry of frames from just one month to get an idea of which tracks cover what area
gf['code'] = gf.relativeOrbit.astype('category').cat.codes
# Extract frames from particular month to get a feel for where frames are
start = '09/01/2019'
end = '09/30/2019'
gdf = gf.query('dateStamp > @start & dateStamp < @end')

In [ ]:
# plot the tracks on our map from before
footprints = gdf.hvplot.polygons(geo=True, hover_cols=['code','relativeOrbit'], cmap='Category10',  
                                 alpha=0.3, legend=True, colorbar=False, title='Sentinel-1 Tracks') #color=hv.Cycle('Category20')
tiles * footprints * aoi * utur 

If you hover over the tracks, a pop-up box displays the orbit/track number.  It looks like tracks 3 and 10 are too far east to be useful, so let's re-make that plot without those tracks.

In [ ]:
tracks = [76,83,149,156] #specify tracks to include
gf = gf[gf.relativeOrbit.isin(tracks)]  #now gf does not include tracks 3 and 10

# display a table summarizing the geometries of the different orbits in gf - now without 3 and 10
gft = gf[['relativeOrbit','geometry']].dissolve(by='relativeOrbit').reset_index()
gft['code'] = gft.relativeOrbit.astype('category').cat.codes # sets a code for orbits
gft.head()

In [ ]:
gf['code'] = gf.relativeOrbit.astype('category').cat.codes
# Extract frames from particular month to get a feel for where frames are
start = '09/01/2019'
end = '09/30/2019'
gdf = gf.query('dateStamp > @start & dateStamp < @end')
# plot the tracks on our map from before
footprints = gdf.hvplot.polygons(geo=True, hover_cols=['code','relativeOrbit'], cmap='Category10',  
                                 alpha=0.3, legend=True, colorbar=False, title='Sentinel-1 Tracks') #color=hv.Cycle('Category20')
tiles * footprints * aoi * utur 

Much better - now we won't be processing a ton of unnecessary data!

Later in the processing we'll specify which sub-swaths of each track to process:


* 156: 1,2
* 149: 2,3
* 76: 1,2
* 83: 2,3

We now have what we need for the next steps in processing:
* "query.geojson" file with the results of the database search
* "apmb.geojson" file with the bounds of the area of interest